# Array
We want to recreate a verion of `np.array`.
## What we'll build
* Similar front-facing behaviour for a tiny portion of numpy array

## What we won't
* We will not make it efficient as numpy in both memroy and runtime considerations

## Rules
* Only use python stdlib

## Array Class

In [1]:
class Arr:
    def __init__(self, data):
        self.data = data

In [2]:
a = Arr([1,2])
print(a)

**Inheriting from same class.**  
You shouldn't do this when writing real code. but for the purpose of making this tutorial more readable, we will define a class that inherits from itself.

In [3]:
class Arr(Arr):
    def __str__(self):
        return (f"Arr: {self.data}")

In [4]:
a = Arr([1,2])
print(a)

Arr: [1, 2]


In [5]:
a

In [6]:
Arr.__repr__ = Arr.__str__

In [7]:
a

Arr: [1, 2]

# Arithmathic

In [8]:
a = Arr([1, 2])
b = Arr([3, 4])

a + b # Will raise Error

TypeError: unsupported operand type(s) for +: 'Arr' and 'Arr'

## The `__add__` method
when `a + b` is called in python. Behind the scenes this is what's called: `a.__add__(b)`

In [10]:
class Arr(Arr):
    def __add__(self, other):
        L = self.data
        R = other.data
        res = []
        for l, r in zip(L,R):
            res.append(l+r)
        return Arr(res)

In [11]:
a = Arr([1, 2])
b = Arr([3, 4])

a + b 

Arr: [4, 6]

In [12]:
c = Arr([10, 20, 30])

Next line works, but it's wrong. `zip` stops iterating when `a` stops.

In [13]:
a + c # Works but wrong

Arr: [11, 22]

In [14]:
d = Arr(['a','b'])

In [15]:
a + d

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [25]:
d + a

TypeError: can only concatenate str (not "int") to str

## A more robust addition function

In [16]:
def array_adder(left, right):
    # Length is the same
    assert len(left) == len(right), f'arrays are not of same shape'
    # Type is the same
    assert dtype(left) == dtype(right), f'Arrays are not of same dtype'
    # We can do the work
    result = [l + r for (l,r) in zip(left,right)]
    return result

In [17]:
array_adder(a,c) # Raises Error

TypeError: object of type 'Arr' has no len()

##  The `__len__` method
Calling `len(x)` class `x.__len__`

In [18]:
class Arr(Arr):
    def __len__(self):
        return len(self.data)

In [19]:
array_adder(Arr([1, 2]), Arr([10, 20]))

NameError: name 'dtype' is not defined

### dtype
An array can only have one dtype. We want to define that, with a few rules:

* On creation, we define the array's dtype
* We will only consider 3 dtypes: `int`, `float` and `str`
* If we get more than 1 type, we will choose the larger type in the hierarchy with `str` > `float` > `int`
* After creation, types stays the same

* Real numpy has a lot more than 3 dtypes:
![hierarchy](https://docs.scipy.org/doc/numpy-1.13.0/_images/dtype-hierarchy.png)

In [20]:
def dtype(obj):
    """Returns the dtype of the array"""
    dtype = int
    for item in obj:
        itype =  type(item)
        if itype == str: # str is the largest, so dtype is str
            return str
        if itype == float: # We haven't seen str by now so type is either float or int
            dtype = float
    return dtype

In [21]:
dtype(Arr([1,2])) # Still doesn't work

TypeError: 'Arr' object is not iterable

> Iterators are everywhere in Python. They are elegantly implemented within for loops, comprehensions, generators etc. but hidden in plain sight.   
>
> Iterator in Python is simply an object that can be iterated upon. An object which will return data, one element at a time.
>
>Technically speaking, Python iterator object must implement two special methods, __iter__() and __next__(), collectively called the iterator protocol.
>
>[Source](https://www.programiz.com/python-programming/iterator)

[Example](98%20-%20Minimals.ipynb#Iterator)

In [25]:
# Implement getitem
class Arr(Arr):
    def __iter__(self):
        return iter(self.data)

We didn't write the `__next__` method. Why?

In [26]:
print (dtype(Arr([1,2])))
print (dtype(Arr([1.,2])))
print (dtype(Arr(['1',2])))

<class 'int'>
<class 'float'>
<class 'str'>


In [27]:
array_adder(Arr([1, 2]), Arr([10, 20]))

[11, 22]

## Convert on init
When `Arr` gets mixed types, we would like it to convert them in the init part.

In [55]:
class Arr(Arr):
    def __init__(self, data):
        self.dtype = dtype(data)
        self.data = [self.dtype(item) for item in data]

## Some Checks

In [56]:
dtype(s1)

str

In [57]:
i1 = Arr([1, 2])
i2 = Arr([3, 4])
f = Arr([3, 4.0])
s1 = Arr([10, 20, '30'])
s2 = Arr(['a', 'b', 'c'])
print(i1 + i2)
print(s1 + s2)

Arr: [4, 6]
Arr: ['10a', '20b', '30c']


## Summary

In [45]:
class Arr:
    def __init__(self, data):
        data = [item for item in data]
        self.dtype = dtype(data)
        self.data = [self.dtype(item) for item in data]

    def __str__(self):
        return (f"Arr: {self.data}")

    __repr__ = __str__

    def __add__(self, other):
        return array_adder(self, other)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.data[item]
    
    def __iter__(self):
        return iter(self.data)


def array_adder(left, right):
    # Length is the same
    assert len(left) == len(right), f'arrays are not of same shape'
    # Type is the same
    assert dtype(left) == dtype(right), f'Arrays are not of same dtype'
    # We can do the work
    result = [l + r for (l, r) in zip(left, right)]
    return result


def dtype(obj):
    """Returns the dtype of the array"""
    dtype = int
    for item in obj:
        itype = type(item)
        if itype == str:  # str is the largest, so dtype is str
            return str
        if itype == float:  # We haven't seen str by now so type is either float or int
            dtype = float
    return dtype

In [44]:
Arr([1,2]) - Arr([100,200])

Arr: [-99, -198]

In [43]:
setattr(Arr, '__sub__', lambda self, other: zip_apply(self, other, getattr(self.dtype, '__sub__')) )

## Exercise
* Make `Arr` have the option to subtract, multiply, divide and make abs values

# Make Arithmetic Generic
Replace `array_adder` by something more robust

In [51]:
# Ex 
def zip_apply(left, right, f):
    # BOE
    # Length is the same
    assert len(left) == len(right), f'arrays are not of same shape'
    # Type is the same
    assert dtype(left) == dtype(right), f'Arrays are not of same dtype'
    # We can do the work
    result = [f(l, r) for (l, r) in zip(left, right)]
    return Arr(result)
    #EOE

This works well, we can try other

In [34]:
class Arr(Arr):
    # BOE
    def __add__(self, other):
        return zip_apply(self, other, self.dtype.__add__)
    
    def __sub__(self, other):
        return zip_apply(self, other, self.dtype.__sub__)

    def __mul__(self, other):
        return zip_apply(self, other, self.dtype.__mul__)

    def __truediv__(self, other):
        return zip_apply(self, other, self.dtype.__truediv__)
    # EOE

In [35]:
print (Arr([1,2]) + Arr([100,200]))
print (Arr([1,2]) * Arr([100,200]))
print (Arr([1,2]) / Arr([100,200]))

Arr: [101, 202]
Arr: [100, 400]
Arr: [0.01, 0.01]


In [36]:
class Arr(Arr):
    def __abs__(self):
        # BOE
        return Arr(map(self.dtype.__abs__, self.data))
        # EOE

In [37]:
abs(Arr([1,-1,-2,5]))

Arr: [1, 1, 2, 5]

# `setattr` and `getattr`

In [58]:
# MATH = ['sub','add','truediv','mul','sub']
# for func in MATH:
#     setattr(Arr, f'__{func}__', lambda self, other: zip_apply(self, other, getattr(self.dtype, f'__{func}__')))